In [ ]:
# Auto reload the notebook for changes in file
%load_ext autoreload
%autoreload 2

In [ ]:
!ls

In [ ]:
!createdb fyyur

In [ ]:
# Initialize the migration for the models existing the app.py
!flask db init

In [ ]:
# Running the first upgrade generated 1e254cd34974_.py alembic file
!flask db migrate

In [ ]:
!flask db upgrade

In [1]:
from data import ven1, ven2, ven3

from app import Venue, Artist, db



In [ ]:
ven1

In [ ]:
v1 = Venue(id = 1, name = ven1["name"], city = ven1["city"], state = ven1["state"], address=ven1["address"],
           phone = ven1["phone"], image_link = ven1["image_link"], 
           facebook_link=ven1["facebook_link"])

In [ ]:
print(v1)

In [ ]:
ven2

In [ ]:
v2 = Venue(id = 2, name = ven2["name"], city = ven2["city"], state = ven2["state"], address=ven2["address"],
           phone = ven2["phone"], image_link = ven2["image_link"], facebook_link=ven2["facebook_link"])

In [ ]:
ven3

In [ ]:
v3 = Venue(id = 3, name = ven3["name"], city = ven3["city"], state = ven3["state"], address=ven3["address"],
           phone = ven3["phone"], image_link = ven3["image_link"], 
           facebook_link=ven3["facebook_link"])

In [ ]:
db.session.add_all([v1, v2, v3])

In [ ]:
db.session.commit()

In [ ]:
# Doing a migration on Artists column genres to array
!flask db migrate

### Migrate
Changing column type did not generate a migration. 
https://alembic.sqlalchemy.org/en/latest/autogenerate.html#what-does-autogenerate-detect-and-what-does-it-not-detect

Changing the column from datatype String to Array has to done on `psql`. Or we remove the string column from the table do a `flask db migrate` and then a `flask db upgrade` and then add the ARRAY datatype column to the table and then migrate.

In [ ]:
!flask db migrate

In [ ]:
!flask db upgrade

In [ ]:
!flask db migrate

In [ ]:
!flask db upgrade

In [ ]:
from data import art1, art2, art3

In [ ]:
a1 = Artist(id = art1["id"], name = art1["name"], city = art1["city"], state = art1["state"],
           phone = art1["phone"], genres = art1["genres"], image_link = art1["image_link"], 
           facebook_link=art1["facebook_link"])

In [ ]:
a2 = Artist(id = art2["id"], name = art2["name"], city = art2["city"], state = art2["state"], 
           phone = art2["phone"], genres = art2["genres"], image_link = art2["image_link"], 
           facebook_link=art2["facebook_link"])

In [ ]:
a3 = Artist(id = art3["id"], name = art3["name"], city = art3["city"], state = art3["state"],
           phone = art3["phone"], genres = art3["genres"], image_link = art3["image_link"]
         
          )

In [ ]:
db.session.add_all([a1, a2, a3])
db.session.commit()

### Adding missing columns to the Artist and Venue tables
<ol>
    <li> Venue </li>
    <ul> 
        <li>Genres</li>
        <li>website</li>
        <li> seeking talent </li>
        <li> seeking description </li>
      </ul>
    <li> Artist </li>
    <ul>
        <li> website </li>
        <li> seeking talent </li>
        <li> seeking description </li>
    </ul>
</ol>

In [ ]:
!flask db migrate

In [ ]:
!flask db upgrade

Now add data to the upgraded table where the values of these new columns are null

In [ ]:
# Changed the tables for artist, venue and also added the show table
!flask db migrate

**Before we run the upgrade always check the auto generated migrated script to make any custom changes** 

In [ ]:
!flask db upgrade

Add data to the show table

In [ ]:
from app import Show

In [ ]:
from data import data

In [ ]:
s1 = Show(show_id=1, venue_id=data[0]['venue_id'], artist_id=data[0]['artist_id'], 
          artist_image_link=data[0]['artist_image_link'], start_time=data[0]['start_time'])

In [ ]:
s2 = Show(show_id=2, venue_id=data[1]['venue_id'], artist_id=data[1]['artist_id'], 
          artist_image_link=data[1]['artist_image_link'], start_time=data[1]['start_time'])

In [ ]:
all_shows = []
for i, d in enumerate(data):
    s = Show(show_id=i+1, venue_id=data[i]['venue_id'], artist_id=data[i]['artist_id'], 
          artist_image_link=data[i]['artist_image_link'], start_time=data[i]['start_time'])
    all_shows.append(s)

In [ ]:
all_shows

In [ ]:
db.session.add_all(all_shows)

In [ ]:
db.session.commit()

#### All the data for the tables has been added ####

In [17]:
from app import format_datetime, Show

In [2]:
import datetime as dt

In [11]:
city_state = Venue.query.with_entities(Venue.city, Venue.state).distinct().all()

In [12]:
print(city_state)

[('San Francisco', 'CA'), ('New York', 'NY')]


In [43]:
data = []
for cs in city_state:
    data.append({'city': cs[0], 'state': cs[1]})
    get_vens_ = Venue.query.filter_by(city=cs[0], state=cs[1]).all()
    # Take the latest element in the list that just got appended
    data[-1]['venues'] = []
    for v in get_vens_:
        
        venue_details = {'id': v.id, 'name': v.name, 'num_upcoming_shows':0}
        
        # Get the shows for this venue
        # This gives the list of shows for this venue
        shows_ = v.show_id # This is the relation that we have already defined in the schema
        # All this can be done without a for loop if we define the column start_time is db.DateTime
        for s in shows_:
            if dt.datetime.strptime(s.start_time, '%Y-%m-%dT%H:%M:%S.%fZ') > dt.datetime.now():
                venue_details['num_upcoming_shows'] += 1        
        data[-1]['venues'].append(venue_details)

In [44]:
data

[{'city': 'San Francisco',
  'state': 'CA',
  'venues': [{'id': 1, 'name': 'The Musical Hop', 'num_upcoming_shows': 0},
   {'id': 3,
    'name': 'Park Square Live Music & Coffee',
    'num_upcoming_shows': 3}]},
 {'city': 'New York',
  'state': 'NY',
  'venues': [{'id': 2,
    'name': 'The Dueling Pianos Bar',
    'num_upcoming_shows': 0}]}]

In [ ]:
print(format_datetime("2019-05-21T21:30:00.000Z"))

In [ ]:
temp_date = format_datetime("2019-05-21T21:30:00.000Z")

In [ ]:
dt_temp_date = dt.datetime.strptime("2019-05-21T21:30:00.000Z", '%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
v1_new = Venue.query.get(1)

In [ ]:
v1_new.genres = ven1['genres']
v1_new.seeking_talent = ven1['seeking_talent']
v1_new.seeking_description = ven1['seeking_description'] 

In [ ]:
db.session.commit()

In [ ]:
v2_new = Venue.query.get(2)
v2_new.genres = ven2['genres']
v2_new.seeking_talent = ven2['seeking_talent']

In [ ]:
v3_new = Venue.query.get(3)
v3_new.genres = ven3['genres']
v3_new.seeking_talent = ven3['seeking_talent']

In [ ]:
db.session.commit()


In [ ]:
a1_new = Artist.query.get(4)

In [ ]:
a1_new.website = art1['website']

In [ ]:
a1_new.seeking_talent = art1['seeking_talent']

In [ ]:
a2_new = Artist.query.get(5)
#a2_new.website = art2['website']
a2_new.seeking_talent = art2['seeking_talent']
a2_new.seeking_description = art2['seeking_description']

In [ ]:
db.session.commit()

In [ ]:
# Running db migrate for changing the name of the column
!flask db migrate

In [ ]:
!flask db upgrade

In [ ]:
a4_new = Artist.query.get(4)

In [ ]:
a4_new.seeking_venue = art1['seeking_venue']

In [ ]:
a4_new.seeking_description = art1['seeking_description']
a4_new.website = art1['website']

In [ ]:
db.session.commit()

In [ ]:
a6_new = Artist.query.get(6)
a6_new.seeking_venue = art3['seeking_venue']

In [ ]:
db.session.commit()

In [ ]:
unique_cities = Venue.query(Venue.city).distinct()

In [ ]:
# test the venue search
ven_search = Venue.query.filter(Venue.name.contains("Music"))

In [ ]:
ven_search.all()

In [ ]:
# ilike needs the pattern matching % refers to any number of characters characters followed (https://www.postgresql.org/docs/9.0/functions-matching.html)
Venue.query.filter(Venue.name.ilike(r"%{0}%".format("music"))).all()

In [ ]:
# get_vens = db.session.query(Venue).distinct(Venue.city, Venue.state)
# get_vens = Venue.query.distinct(Venue.city, Venue.state).group_by(Venue.city, Venue.state)
# get_vens = Venue.query.with_entities(Venue.city, Venue.state).distinct()
get_vens = session.query(Venue.city.distinct()).all()

In [ ]:
get_vens = db.engine.execute('SELECT DISTINCT city, state FROM "Venue";')
# get_vens = Venue.query.group_by(Venue.city Venue.state).all()


In [ ]:
print(get_vens)

In [ ]:
for (city, state) in get_vens:
    print(city, state)
    # get the venue id, name, and num of upcoming shows for each city/state pair
    # vens = Venue.query.filter_by(city=city).order_by('id').all()
    vens = db.engine.execute('''select * from "Venue" where "Venue".city='{0}' and \
                             "Venue".state='{1}' order by "Venue".id;'''.format(city, state))
    for v in vens:
        print(v)
    

In [ ]:
gen_ven_2 = Venue.query.group_by(Venue.id, Venue.city, Venue.state).all()




In [ ]:
print(gen_ven_2)

In [ ]:
city_state[0]

In [ ]:
city_state_2 = Venue.query.with_entities(Venue.city, Venue.state).distinct().all()

In [ ]:
city_state_2

In [ ]:
data

In [ ]:
# Create migration after adding the Show table
!flask db migrate

In [ ]:
!flask db downgrade

In [ ]:
!flask db upgrade